In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras import callbacks

In [2]:
dataFolder = 'C:\Temp\ISIC_Data'
testingFolder = 'C:\Temp\Test_Dataset'
imageWidth = 256
imageHeight = 256
batchSize = 32

In [3]:
trainingDataSet = tf.keras.utils.image_dataset_from_directory(dataFolder, validation_split=0.1111, subset="training", seed=312, batch_size=batchSize, image_size=(imageWidth, imageHeight))

Found 36347 files belonging to 2 classes.
Using 32309 files for training.


In [4]:
validationDataSet = tf.keras.utils.image_dataset_from_directory(dataFolder, validation_split=0.1111, subset="validation", seed=312, batch_size=batchSize, image_size=(imageWidth, imageHeight))

Found 36347 files belonging to 2 classes.
Using 4038 files for validation.


In [5]:
testingDataSet = tf.keras.utils.image_dataset_from_directory(testingFolder, seed=123, batch_size=batchSize, image_size=(imageWidth, imageHeight))

Found 4039 files belonging to 2 classes.


In [6]:
trainingClasses = trainingDataSet.class_names
validationClasses = validationDataSet.class_names
testingClasses = testingDataSet.class_names
print(trainingClasses)
print(validationClasses)
print(testingClasses)
counter = 0
for imageBatch, labelsBatch in trainingDataSet:
    print(imageBatch.shape)
    break
for imageBatch, labelsBatch in validationDataSet:
    print(imageBatch.shape)
    break
for imageBatch, labelsBatch in testingDataSet:
    print(imageBatch.shape)
    break

['BENIGN', 'MALIGNANT_MELANOMA']
['BENIGN', 'MALIGNANT_MELANOMA']
['BENIGN', 'MALIGNANT_MELANOMA']
(32, 256, 256, 3)
(32, 256, 256, 3)
(32, 256, 256, 3)


In [7]:
trainingDataSet = trainingDataSet.cache().shuffle(buffer_size=1000).prefetch(buffer_size=tf.data.AUTOTUNE)
validationDataSet = validationDataSet.cache().shuffle(buffer_size=1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [8]:
print(tf.data.experimental.cardinality(trainingDataSet).numpy())
print(tf.data.experimental.cardinality(validationDataSet).numpy())

1010
127


In [9]:
print(tf.data.experimental.cardinality(trainingDataSet).numpy())
print(tf.data.experimental.cardinality(validationDataSet).numpy())
print(tf.data.experimental.cardinality(testingDataSet).numpy())

1010
127
127


In [10]:
dataAugmentation = Sequential(
    [
        layers.RandomFlip("horizontal_and_vertical", input_shape=(imageWidth, imageHeight, 3)),
        layers.RandomZoom(0.2),
        layers.RandomRotation(0.2)
    ]
)

In [11]:
model = Sequential(
    [
        dataAugmentation,
        layers.Rescaling(1./255),
        layers.Conv2D(16, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(32, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Dropout(0.2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(2)
    ]
)

In [12]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 256, 256, 3)       0         
                                                                 
 rescaling (Rescaling)       (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                  

In [14]:
epochStop = callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=5, restore_best_weights=True)

In [15]:
modelTraining = model.fit(trainingDataSet, validation_data=validationDataSet, epochs=30, callbacks=[epochStop])

Epoch 1/30
1010/1010 [==============================] - 668s 526ms/step - loss: 0.2923 - accuracy: 0.8947 - val_loss: 0.3060 - val_accuracy: 0.8893
Epoch 2/30
1010/1010 [==============================] - 527s 522ms/step - loss: 0.2610 - accuracy: 0.9017 - val_loss: 0.2568 - val_accuracy: 0.9024
Epoch 3/30
1010/1010 [==============================] - 524s 518ms/step - loss: 0.2505 - accuracy: 0.9050 - val_loss: 0.2688 - val_accuracy: 0.8945
Epoch 4/30
1010/1010 [==============================] - 519s 514ms/step - loss: 0.2451 - accuracy: 0.9063 - val_loss: 0.2359 - val_accuracy: 0.9086
Epoch 5/30
1010/1010 [==============================] - 517s 512ms/step - loss: 0.2383 - accuracy: 0.9084 - val_loss: 0.2422 - val_accuracy: 0.9101
Epoch 6/30
1010/1010 [==============================] - 516s 511ms/step - loss: 0.2311 - accuracy: 0.9117 - val_loss: 0.2341 - val_accuracy: 0.9079
Epoch 7/30
1010/1010 [==============================] - 514s 509ms/step - loss: 0.2275 - accuracy: 0.9110 - val_

In [16]:
model.save('C:\Temp\ModelH5\MelanomaModelOptimized.h5')

In [17]:
model.save_weights('C:\Temp\ModelH5\WeightsOptimized\MelanomaModelWeightsOptimized')

In [18]:
modelLoss, modelAccuracy = model.evaluate(testingDataSet)

127/127 [==============================] - 22s 166ms/step - loss: 0.3905 - accuracy: 0.8289


In [19]:
print('Test Accuracy: ', modelAccuracy)
print('Test Loss', modelLoss)

Test Accuracy:  0.8289180397987366
Test Loss 0.3904760479927063
